In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from tensorflow import keras
print(tf.__version__)

2.5.0


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import scipy.stats as stats

In [6]:
data = pd.read_csv('../data/MobiAct/raw_data.csv')
data.head()

,Time,x,y,z,Activity,User
0,235423306000,-0.172383,9.883265,-0.450110,STD,39
1,235450722000,-0.220267,9.921572,-0.440533,STD,39
2,235460429000,-0.258574,9.892841,-0.459687,STD,39
3,235470358000,-0.248997,9.873688,-0.450110,STD,39
4,235490379000,-0.229843,9.825804,-0.411803,STD,39


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5275371 entries, 0 to 5275370
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Time      int64  
 1   x         float64
 2   y         float64
 3   z         float64
 4   Activity  object 
 5   User      int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 241.5+ MB


In [8]:
data.shape

(5275371, 6)

In [9]:
data.isnull().sum()

Time        0
x           0
y           0
z           0
Activity    0
User        0
dtype: int64

In [10]:
data['Activity'].value_counts()

WAL    1729683
STD    1317859
JOG     511585
JUM     501578
STU     305505
STN     302998
CSO     164927
CSI     163639
SCH     152401
SIT      77674
CHU      47522
Name: Activity, dtype: int64

In [11]:
data['x']=data['x'].astype('float')
data['y']=data['y'].astype('float')
data['z']=data['z'].astype('float')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5275371 entries, 0 to 5275370
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Time      int64  
 1   x         float64
 2   y         float64
 3   z         float64
 4   Activity  object 
 5   User      int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 241.5+ MB


In [ ]:
Fs = 20

In [ ]:
activities = data['Activity'].value_counts().index

In [ ]:
activities

In [13]:
df=data.drop(['User','Time'],axis=1).copy()
df.head()

,x,y,z,Activity
0,-0.172383,9.883265,-0.450110,STD
1,-0.220267,9.921572,-0.440533,STD
2,-0.258574,9.892841,-0.459687,STD
3,-0.248997,9.873688,-0.450110,STD
4,-0.229843,9.825804,-0.411803,STD


In [14]:
df['Activity'].value_counts()

WAL    1729683
STD    1317859
JOG     511585
JUM     501578
STU     305505
STN     302998
CSO     164927
CSI     163639
SCH     152401
SIT      77674
CHU      47522
Name: Activity, dtype: int64

In [15]:
label = LabelEncoder()
df['label'] = label.fit_transform(df['Activity'])
df.head()

,x,y,z,Activity,label
0,-0.172383,9.883265,-0.450110,STD,7
1,-0.220267,9.921572,-0.440533,STD,7
2,-0.258574,9.892841,-0.459687,STD,7
3,-0.248997,9.873688,-0.450110,STD,7
4,-0.229843,9.825804,-0.411803,STD,7


In [16]:
label.classes_

array(['CHU', 'CSI', 'CSO', 'JOG', 'JUM', 'SCH', 'SIT', 'STD', 'STN',
       'STU', 'WAL'], dtype=object)

## Standardrizing

In [21]:
X=df[['x','y','z']]
y=df['label']

In [22]:
scaler = StandardScaler()
X= scaler.fit_transform(X)

scaled_X  = pd.DataFrame(data = X,columns = ['x','y','z'])
scaled_X['label']= y.values

scaled_X

,x,y,z,label
0,-0.111117,0.431247,-0.243865,7
1,-0.124153,0.436782,-0.241153,7
2,-0.134581,0.432631,-0.246576,7
3,-0.131974,0.429864,-0.243865,7
4,-0.126760,0.422945,-0.233018,7
...,...,...,...,...
5275366,-0.332724,-2.373387,-0.590948,2
5275367,-0.322296,-2.374770,-0.582814,2
5275368,-0.319689,-2.378921,-0.574679,2
5275369,-0.330117,-2.373387,-0.588237,2


## Frame Preparation

In [23]:
Fs = 20
frame_size = Fs*4 # 80
hop_size = Fs*2 # 40

In [24]:
def get_frames(df, frame_size, hop_size):

    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, len(df) - frame_size, hop_size):
        x = df['x'].values[i: i + frame_size]
        y = df['y'].values[i: i + frame_size]
        z = df['z'].values[i: i + frame_size]
        
        # Retrieve the most often used label in this segment
        label = stats.mode(df['label'][i: i + frame_size])[0][0]
        frames.append([x, y, z])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

In [25]:
X, y = get_frames(scaled_X, frame_size, hop_size)

In [26]:
X.shape,y.shape        #(3555*)/40

((131883, 80, 3), (131883,))

In [27]:
y

array([7, 7, 7, ..., 2, 2, 2])

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

In [64]:
X_train.shape, X_test.shape, y_train.shape

((105506, 80, 3), (26377, 80, 3), (105506,))

In [65]:
X_train[0].shape, X_test[0].shape

((80, 3), (80, 3))

In [67]:
y_train

array([10, 10,  8, ...,  7, 10, 10])

In [69]:
X_train.shape

(105506, 80, 3)

In [70]:
n_steps, n_length = 4, 32

In [71]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[0]

In [72]:
 X_train.shape[1], X_train.shape[2]

(80, 3)

In [73]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(y_train.reshape(-1, 1))

y_train = enc.transform(y_train.reshape(-1, 1))
y_test = enc.transform(y_test.reshape(-1, 1))

In [74]:
y_train.shape

(105506, 11)

In [76]:
X_train.shape

(105506, 80, 3)

In [77]:
n_features #1

3

In [78]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((105506, 80, 3), (105506, 11), (26377, 80, 3), (26377, 11))

In [79]:
verbose, epochs, batch_size = 0, 25, 64
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
# reshape into subsequences (samples, time steps, rows, cols, channels)
n_steps, n_length = 4, 20
X_train = X_train.reshape((X_train.shape[0], n_steps, 1, n_length, n_features))
X_test = X_test.reshape((X_test.shape[0], n_steps, 1, n_length, n_features))
# define model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu')) #input_shape=(n_steps, 1, n_length, n_features)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)
# evaluate model
_, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)

Epoch 1/25
1649/1649 [==============================] - 69s 40ms/step - loss: 0.7375 - accuracy: 0.7659
Epoch 2/25
1649/1649 [==============================] - 66s 40ms/step - loss: 0.5105 - accuracy: 0.8372
Epoch 3/25
1649/1649 [==============================] - 65s 39ms/step - loss: 0.4462 - accuracy: 0.8587
Epoch 4/25
1649/1649 [==============================] - 65s 40ms/step - loss: 0.4044 - accuracy: 0.8723
Epoch 5/25
1649/1649 [==============================] - 66s 40ms/step - loss: 0.3723 - accuracy: 0.8815
Epoch 6/25
1649/1649 [==============================] - 67s 40ms/step - loss: 0.3519 - accuracy: 0.8885
Epoch 7/25
1649/1649 [==============================] - 67s 41ms/step - loss: 0.3316 - accuracy: 0.8954
Epoch 8/25
1649/1649 [==============================] - 67s 41ms/step - loss: 0.3162 - accuracy: 0.8992
Epoch 9/25
1649/1649 [==============================] - 67s 41ms/step - loss: 0.3067 - accuracy: 0.9025
Epoch 10/25
1649/1649 [==============================] - 68s 41m

In [80]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((105506, 4, 1, 20, 3), (105506, 11), (26377, 4, 1, 20, 3), (26377, 11))

In [ ]:
model.summary()

In [81]:
%time y_pred = model.predict_classes(X_test)

/Users/ozge.bursa/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


CPU times: user 11.1 s, sys: 3.95 s, total: 15 s
Wall time: 4.27 s


In [82]:
y_pred

array([10,  3, 10, ..., 10,  7,  7])

In [83]:
np.argmax(y_test, axis=1)

array([10,  3, 10, ..., 10,  7,  7])

In [84]:
label.classes_

array(['CHU', 'CSI', 'CSO', 'JOG', 'JUM', 'SCH', 'SIT', 'STD', 'STN',
       'STU', 'WAL'], dtype=object)

In [ ]:
model.save('../models/mobiact_convlstm2d.h5')

In [85]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((105506, 4, 1, 20, 3), (105506, 11), (26377, 4, 1, 20, 3), (26377, 11))

In [86]:
print(accuracy_score(np.argmax(y_test, axis=1), y_pred))

0.9204610076960988


## TFLITE CONVERTER

In [ ]:
TFLITE_MODEL = '../tflite_models/mobiact/model_keras_lite.tflite'
TFLITE_DYNAMIC_MODEL = '../tflite_models/mobiact/model_keras_lite_dynamic.tflite'
TFLITE_FLOAT16_MODEL = '../tflite_models/mobiact/model_keras_lite_float16.tflite'

In [ ]:
# Convert the model 
converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the SavedModel directory
tflite_model = converter.convert()

In [ ]:
# Save the model.
with open(TFLITE_MODEL, 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Convert the model 
# Dynamic
converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model_dynamic = converter.convert()

In [ ]:
# Save the model.
with open(TFLITE_DYNAMIC_MODEL, 'wb') as f:
  f.write(tflite_quant_model_dynamic)

In [ ]:
# Convert the model 
# Float16
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model_float16 = converter.convert()

In [ ]:
# Save the model.
with open(TFLITE_FLOAT16_MODEL, 'wb') as f:
  f.write(tflite_quant_model_float16)

In [ ]:
'''
def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, 244, 244, 3)
      yield [data.astype(np.float32)]
        
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.representative_dataset = representative_dataset
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]
tflite_quant_model_int_16_8 = converter.convert()
'''

## TFLITE INTERPRETER

## TFLITE

In [ ]:
# Load the TFLite model and allocate tensors.  
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL, num_threads=2)    #(model_content=tflite_model) 
interpreter.allocate_tensors()

In [ ]:
# Get input and output tensors.    
input_det = interpreter.get_input_details()[0]
output_det = interpreter.get_output_details()[0]

In [ ]:
input_det

In [ ]:
output_det

In [ ]:
input_index = input_det["index"]
output_index = output_det["index"]
input_shape = input_det["shape"]
output_shape = output_det["shape"]
input_dtype = input_det["dtype"]
output_dtype = output_det["dtype"]

In [ ]:
input_index, output_index, input_shape, output_shape, input_dtype, output_dtype

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
def predict(inp, input_dtype, output_shape, output_dtype, interpreter, input_index, output_index):
    inp = inp.astype(input_dtype)
    count = inp.shape[0]
    out = np.zeros((count, output_shape[1]), dtype=output_dtype)
    for i in range(count):
        interpreter.set_tensor(input_index, inp[i:i+1])
        interpreter.invoke()
        out[i] = interpreter.get_tensor(output_index)[0]
    return out

In [ ]:
%time y_pred_lite = predict(X_test, input_dtype, output_shape, output_dtype, interpreter, input_index, output_index)

In [ ]:
y_pred_lite.shape

In [ ]:
y_pred_lite = np.argmax(y_pred_lite, axis=1) 
y_pred_lite

## TFLITE DYNAMIC

In [ ]:
# Load the TFLite model and allocate tensors.  
interpreter = tf.lite.Interpreter(model_path=TFLITE_DYNAMIC_MODEL, num_threads=4)    #(model_content=tflite_model) 
interpreter.allocate_tensors()

# Get input and output tensors.    
input_det = interpreter.get_input_details()[0]
output_det = interpreter.get_output_details()[0]

input_index = input_det["index"]
output_index = output_det["index"]
input_shape = input_det["shape"]
output_shape = output_det["shape"]
input_dtype = input_det["dtype"]
output_dtype = output_det["dtype"]

In [ ]:
%time y_pred_lite_dynamic = predict(X_test, input_dtype, output_shape, output_dtype, interpreter, input_index, output_index)

In [ ]:
y_pred_lite_dynamic = np.argmax(y_pred_lite_dynamic, axis=1) 
y_pred_lite_dynamic

## TFLITE FLOAT16

In [ ]:
# Load the TFLite model and allocate tensors.  
interpreter = tf.lite.Interpreter(model_path=TFLITE_FLOAT16_MODEL, num_threads=2)    #(model_content=tflite_model) 
interpreter.allocate_tensors()

# Get input and output tensors.    
input_det = interpreter.get_input_details()[0]
output_det = interpreter.get_output_details()[0]

input_index = input_det["index"]
output_index = output_det["index"]
input_shape = input_det["shape"]
output_shape = output_det["shape"]
input_dtype = input_det["dtype"]
output_dtype = output_det["dtype"]

In [ ]:
input_shape

In [ ]:
%time y_pred_lite_float16 = predict(X_test, input_dtype, output_shape, output_dtype, interpreter, input_index, output_index)

In [ ]:
y_pred_lite_float16 = np.argmax(y_pred_lite_float16, axis=1) 
y_pred_lite_float16

In [ ]:

with tf.device('/gpu:0'):
    %time y_pred_lite_float16 = predict(X_test, input_dtype, output_shape, output_dtype, interpreter, input_index, output_index)


In [ ]:

with tf.device('/cpu:0'):
    %time y_pred_lite_float16 = predict(X_test, input_dtype, output_shape, output_dtype, interpreter, input_index, output_index)


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Ploting Confusion Matrix

model.save_weights('cnn_model.h5')

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
print(accuracy_score(y_test, y_pred_lite))

In [ ]:
print(accuracy_score(y_test, y_pred_lite_dynamic))

In [ ]:
print(accuracy_score(y_test, y_pred_lite_float16))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
def plot_cm_new(y_true, y_pred, class_names):
  cm = confusion_matrix(y_true, y_pred)
  fig, ax = plt.subplots(figsize=(18, 16)) 
  ax = sns.heatmap(
      cm, 
      annot=True, 
      fmt="d", 
      cmap=sns.color_palette("muted"),
      ax=ax
  )

  plt.ylabel('Actual')
  plt.xlabel('Predicted')
  ax.set_xticklabels(class_names)
  ax.set_yticklabels(class_names)
  b, t = plt.ylim() # discover the values for bottom and top
  b += 0.5 # Add 0.5 to the bottom
  t -= 0.5 # Subtract 0.5 from the top
  plt.ylim(b, t) # update the ylim(bottom, top) values
  #plt.show() # ta-da!
  plt.savefig("confusion_matrix_cnn_im.png")

In [ ]:
plot_cm_new(
  label.inverse_transform(y_test),
  label.inverse_transform(y_pred),
  label.classes_
)

In [ ]:
label.classes_

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(label.inverse_transform(y_test), label.inverse_transform(y_pred)))
# Accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(label.inverse_transform(y_test), label.inverse_transform(y_pred)))
# Recall
from sklearn.metrics import recall_score
print(recall_score(label.inverse_transform(y_test), label.inverse_transform(y_pred), average=None))
# Precision
from sklearn.metrics import precision_score
print(precision_score(label.inverse_transform(y_test), label.inverse_transform(y_pred), average=None))
# F1 score
from sklearn.metrics import f1_score
f1_score(label.inverse_transform(y_test), label.inverse_transform(y_pred), average=None)